In [1]:
# RED NEURONAL ARTIFICIAL (RNA) _ PARA GANAR JUGANDO A PIEDRA, PAPEL O TIJERA

In [2]:
options=["piedra","tijeras","papel"]# Opciones posibles

In [3]:
def search_winner(p1,p2): #Definir quién gana
    if p1==p2:
        result=0    #Definir empates
    elif p1=="piedra" and p2=="tijeras":
        result=1
    elif p1=="piedra" and p2=="papel":
        result=2
    elif p1=="tijeras" and p2=="piedra":
        result=2
    elif p1=="tijeras" and p2=="papel":
        result=1
    elif p1=="papel" and p2=="piedra":
        result=1
    elif p1=="papel" and p2=="tijeras":
        result=2 #Definir todos los no empates
    return result

In [4]:
search_winner("papel","tijeras") # Prueba de que funciona bien

2

In [5]:
test=[
    ["piedra","piedra",0],
    ["piedra","tijeras",1],
    ["piedra","papel",2]
]
for partida in test:
    print("player1: %s player2: %s Winner:%s Validation: %s "%(
        partida[0],partida[1],search_winner(partida[0],partida[1]),partida[2]
    ))

player1: piedra player2: piedra Winner:0 Validation: 0 
player1: piedra player2: tijeras Winner:1 Validation: 1 
player1: piedra player2: papel Winner:2 Validation: 2 


In [6]:
# Prueba de la elección aleatoria
from random import choice 
def get_choice():
    return choice(options)

In [7]:
for i in range(10): 
    player1=get_choice()
    player2=get_choice()
    print("player1: %s player2: %s Winner: %s "%(
        player1,player2,search_winner(player1,player2)
    ))

player1: piedra player2: piedra Winner: 0 
player1: piedra player2: papel Winner: 2 
player1: piedra player2: tijeras Winner: 1 
player1: tijeras player2: papel Winner: 1 
player1: tijeras player2: papel Winner: 1 
player1: piedra player2: piedra Winner: 0 
player1: papel player2: tijeras Winner: 2 
player1: piedra player2: tijeras Winner: 1 
player1: papel player2: tijeras Winner: 2 
player1: papel player2: papel Winner: 0 


In [8]:
# Convertimos los stream en 0 y 1 porque le va mejor a la RNA

In [9]:
def str_to_list(option):
    if option=="piedra":
        res=[1,0,0]
    elif option=="tijeras":
        res=[0,1,0]
    else:
        res=[0,0,1]
    return res 
data_X=list(map(str_to_list,["piedra","tijeras","papel"]))
data_y=list(map(str_to_list,["papel","piedra","tijeras"]))
# y va a aprender porque le hemos dicho qué gana al player1
print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [10]:
from sklearn.neural_network import MLPClassifier

In [11]:
# Para comprobar que de verdad aprende, sólo le enseñamos que 
# la piedra pierde con el papel, que es lo almacenado en data_X y data_y
# en la posición 0. El resto tendrá que descubrirlo.

In [12]:
clf=MLPClassifier(verbose=False, warm_start=True)
model=clf.fit([data_X[0]], [data_y[0]])
print(model)

MLPClassifier(warm_start=True)


C:\Users\eduma\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [13]:
# Vamos a programar que player1 coge una opción random y haremos que# nuestro algoritmo tenga que elegir.
# Le hemos enseñado una opción y tiene que aprender las otras dos

In [16]:
def play_and_learn (iters=10,debug=False):
    score={"win":0,"lose":0}# variable para guardar la estadística
    data_X=[]
    data_y=[]
    for i in range(iters):
        player1=get_choice()
        
        predict=model.predict_proba([str_to_list(player1)])[0]
        # Elije con una probabilidad determinada de ganar
        
        if predict[0]>=0.95:
            player2=options[0]
        elif predict[1]>=0.95:
            player2=options[1]
        elif predict[2]>=0.95:
            player2=options[2]
        else:
            player2=get_choice() # Si ninguna da un 95% coge al azar
        
        
        
        if debug==True:
            print("Player1: %s Player2(modelo): %s -->%s"%
                  (player1,predict,player2))# Para ver que hace
            
# Nos interesa ahora decirle al modelo cuando gana para que aprenda        
        winner=search_winner(player1,player2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s"%winner)
            
# Si ha ganado, se guarda la opción ganadora para la próxima vez
        if winner==2:
            data_X.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            score["win"]+=1
        else:
            score["lose"]+=1
    return score,data_X,data_y

In [18]:
score,data_X,data_y=play_and_learn(1,debug=True)
print(data_X)
print(data_y)
print("Score: %s%s%%"%(score,(score["win"]*100/(score["win"]+score["lose"]))))
# Es para hallar los porcentajes de victorias
if len(data_X):
    model=model.partial_fit(data_X,data_y)

Player1: tijeras Player2(modelo): [0.13258855 0.10975204 0.88934778] -->piedra
Comprobamos: p1 VS p2: 2
[[0, 1, 0]]
[[1, 0, 0]]
Score: {'win': 1, 'lose': 0}100.0%


In [19]:
# Se repite muchas veces hasta que aprenda a ganar
# Generamos un histórico de partidas

In [20]:
i=0
historic_pct=[]
while True:
    i+=1
    score,data_X,data_y=play_and_learn(1000,debug=False)
    pct=(score["win"]*100/(score["win"]+score["lose"]))
    historic_pct.append(pct)# añadimos al histórico el porcentaje corriente para ver en una lista como mejora
    
    print("iter: %s - score: %s%s%%"%(i,score,pct))
    if len(data_X):
        model=model.partial_fit(data_X,data_y)
        
    if sum (historic_pct[-9:])==900:
        break
    # Si la suma de las últimas nueve partidas es 900, significa que ha sacado 100% en todas...entonces para. 
    from bokeh.plotting import figure,show
    from bokeh.io import push_notebook,show,output_notebook
    output_notebook()

iter: 1 - score: {'win': 571, 'lose': 429}57.1%


Loading BokehJS ...

iter: 2 - score: {'win': 578, 'lose': 422}57.8%


Loading BokehJS ...

iter: 3 - score: {'win': 542, 'lose': 458}54.2%


...

iter: 200 - score: {'win': 1000, 'lose': 0}100.0%


Loading BokehJS ...

iter: 201 - score: {'win': 1000, 'lose': 0}100.0%


Loading BokehJS ...

iter: 202 - score: {'win': 1000, 'lose': 0}100.0%


In [23]:
x=range(len(historic_pct))
y=historic_pct
p=figure(title="Porcentaje de aprendizaje en cada iteración",x_axis_label="Iter",y_axis_label="%",width=900)
p.line(x,y,line_width=1)
show(p)